In [1]:
import os
gpu_number = "0" # Choose either 0 or 1
os.environ['CUDA_ENVIRONMENT_DEVICES'] = gpu_number

In [2]:
import os
import sys
import numpy as np
from numpy import asarray,zeros
import pandas as pd 
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt
import copy
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
import transformers
from transformers import BertTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel, AutoConfig, AdamW, get_linear_schedule_with_warmup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import timm
from losses import SupConLoss
from torchlars import LARS

In [3]:
if torch.cuda.is_available():
    device_name = "cuda:" + gpu_number
    device = torch.device(device_name)
else:
    device = torch.device("cpu")
# device = torch.device("cpu") # Force CPU
print("Using device", device)

Using device cuda:0


In [4]:
# NOTE: Resnet+BERT Multimodal data
source_multimodal_arr = np.load("../data/source_multimodal_out.npy")
target_multimodal_arr = np.load("../data/target_multimodal_out.npy")
print("Source shape", source_multimodal_arr.shape)
print("Target shape", target_multimodal_arr.shape)

Source shape (11766, 1280)
Target shape (11766, 1280)


In [5]:
# # NOTE: Import Visual BERT Multimodal data
# source_multimodal_arr = np.load("../data/source_mm_vbert.npy")
# target_multimodal_arr = np.load("../data/target_mm_vbert.npy")

In [6]:
# NOTE: Import source data for emotion
# img_data = np.load("../data/image_array.npy")
# num_images, sources, width, height, num_channels = img_data.shape
# img_data_reshape = np.reshape(img_data, newshape=(num_images, sources, num_channels, width, height))
# img_data_source = torch.tensor(img_data_reshape[:,1,:,:,:]) 
# print('New Source Shape', img_data_source.shape)
emotion_arr = np.load('../data/emotion_reprs.npy')
print('Emotion array shape', emotion_arr.shape)

Emotion array shape (11766, 128)


In [7]:
# # New features
# diff_arr = source_multimodal_arr - target_multimodal_arr
# mul_diff_arr = source_multimodal_arr * diff_arr
# print("Source shape", diff_arr.shape)
# print("Target shape", mul_diff_arr.shape)

In [8]:
# # Considering only the text parts - #TODO: Comment if not necessary
# source_multimodal_arr = source_multimodal_arr[:,768:] # Only BERT
# target_multimodal_arr = target_multimodal_arr[:,768:] # Only BERT
# print("Source shape", source_multimodal_arr.shape)
# print("Target shape", target_multimodal_arr.shape)

In [9]:
# NOTE: Old features
source_multimodal_tensor = torch.tensor(source_multimodal_arr)
target_multimodal_tensor = torch.tensor(target_multimodal_arr)
emotion_source_tensor = torch.tensor(emotion_arr)
print("Source shape", source_multimodal_tensor.shape)
print("Target shape", target_multimodal_tensor.shape)
print("Emotion Source shape", emotion_source_tensor.shape)

Source shape torch.Size([11766, 1280])
Target shape torch.Size([11766, 1280])
Emotion Source shape torch.Size([11766, 128])


In [10]:
# #NOTE: New features (NOT GOOD)
# diff_arr = torch.tensor(diff_arr, device=device)
# mul_diff_arr = torch.tensor(mul_diff_arr, device=device)
# print("Source shape", diff_arr.shape)
# print("Target shape", mul_diff_arr.shape)

In [11]:
# Loading the label
labels_data = np.load("../data/labels.npy")
labels_tensor = torch.tensor(labels_data, dtype=torch.long).squeeze(-1)
print('Labels tensor shape', labels_tensor.shape)

Labels tensor shape torch.Size([11766])


In [12]:
#TODO: Add Pytorch DataLoader
def get_data_loader(batch_size, target_input, source_input, emotion_input, labels):
	data = TensorDataset(target_input, source_input, emotion_input, labels)
	sampler = SequentialSampler(data)
	dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)
	return data, sampler, dataloader

In [13]:
# NOTE: Old features
train_target, test_target, train_source, test_source, train_emotion, test_emotion, train_labels, test_labels = train_test_split(target_multimodal_tensor, source_multimodal_tensor, emotion_source_tensor, labels_tensor, test_size=0.2, random_state=43)

In [14]:
batch_size = 128
train_data, train_sampler, train_dataloader = get_data_loader(batch_size, train_target, train_source, train_emotion, train_labels)
test_data, test_sampler, test_dataloader = get_data_loader(batch_size, test_target, test_source, test_emotion, test_labels)

In [15]:
# Model imported from the previous network
class ContrastiveModel(nn.Module):
    def __init__(self, initial_dim):
        super(ContrastiveModel, self).__init__()
        self.project_1 = nn.Linear(initial_dim, 512, bias=True)
        self.project_2 = nn.Linear(512, 128, bias=True)
        self.dropout = nn.Dropout(0.2)
    def forward(self, multimodal_input):
        contrast_space = self.project_2(self.project_1(multimodal_input))
        normalize_contrast = F.normalize(contrast_space, dim=2)
        return normalize_contrast

In [16]:
# NOTE: Novelty based Contrastive Model
class NoveltyModule(nn.Module):
    def __init__(self, initial_dim):
        super(NoveltyModule, self).__init__()
        self.contrastive_model = ContrastiveModel(initial_dim) # For generic model
        # self.contrastive_model = ContrastiveModelVisualBERT(initial_dim) # For visual BERT only
        # self.contrastive_model.load_state_dict(torch.load('saved_models/contrast_head_visualbert_lr_5_ep_1000.pt')) # For vbert
        self.contrastive_model.load_state_dict(torch.load('saved_models/contrast_head_lr_5_ep_1000_old.pt')) # For rbert
        for param in self.contrastive_model.parameters():
            param.requires_grad = False
        self.reduce_1 = nn.Linear((512)*3, 512)
        self.tanh1 = nn.Tanh()
        self.logits_layer = nn.Linear(512, 2)
    def forward(self, target_input, source_input):
        fixed_target = self.contrastive_model.project_1(target_input)
        fixed_source = self.contrastive_model.project_1(source_input)
        add_op = fixed_target + fixed_source
        sub_op = fixed_target - fixed_source
        mul_op = fixed_target * fixed_source
        combine_t_s = torch.cat((add_op, sub_op, mul_op), 1)
        reduce_output = self.tanh1(self.reduce_1(combine_t_s)) # 512 dimension
        return reduce_output

In [17]:
# NOTE: Emotion based Model
class ResNetBottom(nn.Module):
    def __init__(self, original_model):
        super(ResNetBottom, self).__init__()
        self.features = nn.Sequential(*list(original_model.children())[:-1])
        
    def forward(self, x):
        x = self.features(x)
        return x

class EmotionModule(nn.Module):
    def __init__(self):
        super(EmotionModule, self).__init__()
        self.vision_base_model = timm.create_model('resnet50', pretrained=True)
        self.vision_model_head = ResNetBottom(self.vision_base_model)
        self.project_1 = nn.Linear(2048, 1024, bias=True)
        self.project_2 = nn.Linear(1024, 512, bias=True)
        self.project_3 = nn.Linear(512, 128, bias=True)
        self.tanh1 = nn.Tanh()
        self.tanh2 = nn.Tanh()
        self.tanh3 = nn.Tanh()
        self.drop1 = nn.Dropout()
        self.drop2 = nn.Dropout()
        self.drop3 = nn.Dropout()
        self.classification = nn.Linear(128, 2, bias=True)
    def forward(self, img_features):
        with torch.no_grad():
            img_out = self.vision_model_head(img_features)
        emotion_features = self.tanh3(self.project_3(self.tanh2(self.project_2(self.tanh1(self.project_1(img_out))))))
        return emotion_features

In [18]:
#TODO: Defin Combine Novelty-Emotion Model for classification
class FinalClassifier(nn.Module):
    def __init__(self, initial_dim):
        super(FinalClassifier, self).__init__()
        self.novelty_module = NoveltyModule(initial_dim) # For novelty model
        # self.emotion_module = EmotionModule()
        # self.emotion_module.load_state_dict(torch.load('../emotion/saved_models/emo_combine_res50_lr_3e-05_val_loss_0.59487_ep_61.pt'))
        self.reduce_1 = nn.Linear(640, 512, bias=True)
        self.tanh1 = nn.Tanh()
        self.reduce_2 = nn.Linear(512, 128, bias=True)
        self.tanh2 = nn.Tanh()
        self.logits_layer = nn.Linear(128, 2)
    def forward(self, target_input, source_input, emotion_input):
        # Novelty Representations
        novelty_reprs = self.novelty_module(target_input, source_input)
        emotion_reprs = emotion_input
        combine_n_e = torch.cat((novelty_reprs, emotion_reprs), 1) # 640
        reduce_output_1 = self.tanh1(self.reduce_1(combine_n_e)) # 512 dimension
        reduce_output_2 = self.tanh2(self.reduce_2(reduce_output_1)) # 128 dimension
        logits = self.logits_layer(reduce_output_2) # 2 dimension
        return logits

In [19]:
# Dry-Run Classification model
initial_dim = source_multimodal_tensor.shape[1]
# initial_dim = diff_arr.shape[1]
class_model = FinalClassifier(initial_dim).to(device) # For old model
# class_model = ContrastiveClassificationNew(initial_dim).to(device) # For new model
# class_out = class_model(target_multimodal_tensor[0:32])
# print(class_out.shape)

In [20]:
# Optimizer and scheduler
def get_optimizer_scheduler(name, model, train_dataloader_len, epochs, lr_set):
	if name == "Adam":
		optimizer = AdamW(model.parameters(),
                  lr = lr_set, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
		)
	elif name == "LARS-SGD":
		base_optimizer = optim.SGD(model.parameters(), lr=lr_set, momentum=0.9)
		optimizer = LARS(optimizer=base_optimizer, eps=1e-8, trust_coef=0.001)

	total_steps = train_dataloader_len * epochs

	# Create the learning rate scheduler.
	scheduler = get_linear_schedule_with_warmup(optimizer, 
												num_warmup_steps = total_steps//2, # Default value in run_glue.py
												num_training_steps = total_steps)
	return optimizer, scheduler

In [21]:
# Getting the optimizer and scheduler
epochs = 100
lr = 3e-5 # Less LR
# lr = 0.5
iters_to_accumulate = 2
name = "Adam"
# name = "LARS-SGD"
criterion = nn.CrossEntropyLoss()
optimizer, scheduler = get_optimizer_scheduler(name, class_model, len(train_dataloader), epochs, lr)

In [22]:
################ Evaluating Loss ######################
#######################################################
def evaluate_loss(net, device, criterion, dataloader):
    net.eval()
    mean_loss = 0
    count = 0
    with torch.no_grad():
        for it, (target_inputs, source_inputs, emotion_inputs, labels) in enumerate(tqdm(dataloader)):
            target_inputs, source_inputs, emotion_inputs, labels = target_inputs.to(device), source_inputs.to(device), emotion_inputs.to(device), labels.to(device)
            logits = net(target_inputs, source_inputs, emotion_inputs)
            mean_loss += criterion(logits.squeeze(-1), labels).item() # initially it was logits.squeeze(-1)
            count += 1
    return mean_loss / count

In [23]:
################ Flat Accuracy Calculation ####################
###############################################################
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
################ Validation Accuracy Calculation ####################
###############################################################
def evaluate_accuracy(model, device, validation_dataloader):
	model.eval()
	# Tracking variables 
	eval_loss, eval_accuracy = 0, 0
	nb_eval_steps, nb_eval_examples = 0, 0
	# Evaluate data for one epoch
	for batch in validation_dataloader:
	    # Add batch to GPU
	    batch = tuple(t.to(device) for t in batch)	    
	    # Unpack the inputs from our dataloader
	    b_t_inputs, b_s_inputs, b_e_inputs, b_labels = batch	    
	    
	    # Telling the model not to compute or store gradients, saving memory and
	    # speeding up validation
	    with torch.no_grad(): 
	    	# Forward pass, calculate logit predictions.
	        # This will return the logits rather than the loss because we have
	        # not provided labels.
	    	logits = model(b_t_inputs, b_s_inputs, b_e_inputs)       

	    # Move logits and labels to CPU
	    logits = logits.detach().cpu().numpy()
	    label_ids = b_labels.to('cpu').numpy()
	    
	    # Calculate the accuracy for this batch of test sentences.
	    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
	    
	    # Accumulate the total accuracy.
	    eval_accuracy += tmp_eval_accuracy

	    # Track the number of batches
	    nb_eval_steps += 1
	accuracy = eval_accuracy/nb_eval_steps
	return accuracy

In [24]:
def train_model(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate):
    best_loss = np.Inf
    best_ep = 1
    nb_iterations = len(train_loader)
    print_every = nb_iterations // 5  # print the training loss 5 times per epoch
    iters = []
    train_losses = []
    val_losses = []
    # Iterating over all epochs
    for ep in range(epochs):
        net.train()
        running_loss = 0.0
        for it, (target_inputs, source_inputs, emotion_inputs, labels) in enumerate(tqdm(train_loader)):

            # Converting to cuda tensors
            target_inputs, source_inputs, emotion_inputs, labels = target_inputs.to(device), source_inputs.to(device), emotion_inputs.to(device), labels.to(device)
    		
            # Obtaining the logits from the model
            logits = net(target_inputs, source_inputs, emotion_inputs)
            # print(logits.device)

            # Computing loss
            # print(logits.squeeze(-1).shape)
            # print(labels.shape)
            loss = criterion(logits.squeeze(-1), labels)
            loss = loss / iters_to_accumulate  # Normalize the loss because it is averaged

            # Backpropagating the gradients
            # Calls backward()
            loss.backward()

            if (it + 1) % iters_to_accumulate == 0:
                # Optimization step
                # scaler.step() first unscales the gradients of the optimizer's assigned params.
                # If these gradients do not contain infs or NaNs, opti.step() is then called,
                # otherwise, opti.step() is skipped.
                opti.step()
                # Adjust the learning rate based on the number of iterations.
                lr_scheduler.step()
                # Clear gradients
                net.zero_grad()


            running_loss += loss.item()

            if (it + 1) % print_every == 0:  # Print training loss information
                print()
                print("Iteration {}/{} of epoch {} complete. Loss : {} "
                      .format(it+1, nb_iterations, ep+1, running_loss / print_every))

                running_loss = 0.0


        val_loss = evaluate_loss(net, device, criterion, val_loader)  # Compute validation loss
        val_accuracy = evaluate_accuracy(net, device, val_loader)
        print()
        print("Epoch {} complete! Validation Loss : {}".format(ep+1, val_loss))
        print("Epoch {} complete! Validation Accuracy : {}".format(ep+1, val_accuracy))

        if val_loss < best_loss:
            print("Best validation loss improved from {} to {}".format(best_loss, val_loss))
            print()
            net_copy = copy.deepcopy(net)  # save a copy of the model
            best_loss = val_loss
            best_ep = ep + 1

    # Saving the model
    model_name = 'ne_full_nv_em_full_rbert'
    path_to_model='saved_models/{}_lr_{}_val_loss_{}_ep_{}.pt'.format(model_name, lr, round(best_loss, 5), best_ep)
    torch.save(net_copy.state_dict(), path_to_model)
    net.load_state_dict(torch.load(path_to_model)) # Re-Loading the best model
    print("The model has been saved in {}".format(path_to_model))

    del loss
    torch.cuda.empty_cache()
    return net

In [25]:
# Train the model
model = train_model(class_model, criterion, optimizer, lr, scheduler, train_dataloader, test_dataloader, epochs, iters_to_accumulate)

 54%|█████▍    | 40/74 [00:00<00:00, 126.58it/s]


Iteration 14/74 of epoch 1 complete. Loss : 0.3629037141799927 

Iteration 28/74 of epoch 1 complete. Loss : 0.36374300505433765 


 92%|█████████▏| 68/74 [00:00<00:00, 132.00it/s]


Iteration 42/74 of epoch 1 complete. Loss : 0.36088602031980244 

Iteration 56/74 of epoch 1 complete. Loss : 0.36058466136455536 


  0%|          | 0/74 [00:00<?, ?it/s]

...

 76%|███████▌  | 56/74 [00:00<00:00, 135.21it/s]


Iteration 42/74 of epoch 100 complete. Loss : 0.0008175605325959623 

Iteration 56/74 of epoch 100 complete. Loss : 0.0008147472565594528 


100%|██████████| 19/19 [00:00<00:00, 236.43it/s]


Iteration 70/74 of epoch 100 complete. Loss : 0.0008237751816133303 

Epoch 100 complete! Validation Loss : 0.2814306334445351
Epoch 100 complete! Validation Accuracy : 0.9238322368421054


The model has been saved in saved_models/ne_full_nv_em_full_rbert_lr_3e-05_val_loss_0.19496_ep_62.pt


In [26]:
def evaluate(prediction_dataloader, model, path_to_model, load = False):
	# Prediction on test set
	if load:
		print("Loading the weights of the model...")
		model.load_state_dict(torch.load(path_to_model))

	print('Evaluating on the testset')

	# Put model in evaluation mode
	model.eval()

	# Tracking variables 
	predictions , true_labels = [], []

	# Predict 
	for batch in prediction_dataloader:
	  # Add batch to GPU
	  batch = tuple(t.to(device) for t in batch)
	  
	  # Unpack the inputs from our dataloader
	  b_t_inputs, b_s_inputs, b_e_inputs, b_labels = batch
	  
	  # Telling the model not to compute or store gradients, saving memory and 
	  # speeding up prediction
	  with torch.no_grad():
	      # Forward pass, calculate logit predictions
	      logits = model(b_t_inputs, b_s_inputs, b_e_inputs)

	  # Move logits and labels to CPU
	  logits = logits.detach().cpu().numpy()
	  label_ids = b_labels.to('cpu').numpy()

	  pred_flat = np.argmax(logits, axis=1).flatten()
	  labels_flat = label_ids.flatten()
	  
	  # Store predictions and true labels
	  predictions.extend(pred_flat)
	  true_labels.extend(labels_flat)

	print('DONE.')

	# Code for result display
	print('NovFake Multi-Modal Classification accuracy is')
	print(metrics.accuracy_score(true_labels, predictions)*100)
	print(classification_report(true_labels, predictions, target_names = ['fake', 'real']))
	# Converting to csv
	# Removed transpose - check if actually required
	model_name = 'nv_em_full_rbert'
	clsf_report = pd.DataFrame(classification_report(y_true = true_labels, y_pred = predictions, output_dict=True, target_names = ['fake', 'real']))
	clsf_report.to_csv(str('saved_models/'+model_name+'.csv'), index= True)

In [27]:
path_to_model = 'saved_models/class_contrast_visualbert_lr_3e-05_val_loss_0.35285_ep_100.pt'
evaluate(test_dataloader, model, path_to_model = path_to_model, load = False)

Evaluating on the testset
DONE.
NovFake Multi-Modal Classification accuracy is
92.01359388275276
              precision    recall  f1-score   support

        fake       0.92      0.90      0.91      1031
        real       0.92      0.94      0.93      1323

    accuracy                           0.92      2354
   macro avg       0.92      0.92      0.92      2354
weighted avg       0.92      0.92      0.92      2354

